In [1]:
from datetime import datetime as dt  #to manipulate dates

import numpy as np  #to cleanse data
import openpyxl
import pandas as pd
import plotly.express as px  #to create interactive charts
import plotly.graph_objects as go  #to create interactive charts
from datetime import datetime

In [2]:
dtype_dict = {
    'Description': 'string',
    'Amount': 'float',
    'Type': 'string',
    'Current balance': 'float',
    'Status': 'string'
}

ytd_data = pd.read_csv(
    'D:\\Sean\\Documents\\Personal\\Budget\\YTD_downloads\\2025\\SOFI-Savings.csv', dtype=dtype_dict, parse_dates=['Date']
)


ytd_data['Account'] = 'sofi_savings'
ytd_data.rename(columns={'Date':'Trans. Date','Current balance':'CumAmount'}, inplace=True)

ytd_data['Date'] = pd.to_datetime(ytd_data['Trans. Date'].dt.strftime('%m/%Y'), format='%m/%Y')

In [3]:
ytd_data

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date
0,2025-10-22,UNITED FIN CAS,Direct Payment,-86.33,4056.51,Posted,sofi_savings,2025-10-01
1,2025-10-21,To Checking - 8930,Withdrawal,-1000.00,4142.84,Posted,sofi_savings,2025-10-01
2,2025-10-18,To Checking - 8930,Withdrawal,-1000.00,5142.84,Posted,sofi_savings,2025-10-01
3,2025-10-16,DISCOVER,Direct Payment,-1004.32,6142.84,Posted,sofi_savings,2025-10-01
4,2025-10-14,Nationwide,Direct Deposit,3140.74,7147.16,Posted,sofi_savings,2025-10-01
...,...,...,...,...,...,...,...,...
103,2025-01-13,To Checking - 8930,Withdrawal,-155.00,5938.22,Posted,sofi_savings,2025-01-01
104,2025-01-12,From Checking - 8930,Deposit,1000.00,6093.22,Posted,sofi_savings,2025-01-01
105,2025-01-07,To Checking - 8930,Withdrawal,-1634.00,5093.22,Posted,sofi_savings,2025-01-01
106,2025-01-07,Nationwide,Direct Deposit,2101.68,6727.22,Posted,sofi_savings,2025-01-01


In [10]:
# Discover Transactions
mapping = {
    # Direct Deposit
    'income': ['PAYROLL', "Nationwide",'Franchise Tax BD', 'IRS  TREAS 310'],
    'interest': ['INTEREST'],
    'school':['BOARD OF REGENTS'],
    'transfer' :['TRANSFER','To Checking - 8930','Discover','WELLS FARGO BANK','WELLS FARGO IFI'],
    'Don': ['Fortiva'],
    'transfer': ['From Checking'],
    'progressive': ['UNITED FIN CAS'],
    'refund': ['HEALTHEQUITY INC','TICKETMASTER RES'],
    'school': ['GEORGIA TECH ACH']

    
    
}
for k, v in mapping.items():
    ytd_data.loc[ytd_data.Description.str.contains('|'.join(v), case=False), 'Short_Desc'] = k
    ytd_data.loc[ytd_data.Description.str.contains("WELLS FARGO BANK", case=False) & (
            ytd_data['Amount'] == -2100), 'Short_Desc'] = 'family'


In [11]:
ytd_data[ytd_data['Short_Desc'].isnull()].sort_values(by='Amount', ascending=False)

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date,Short_Desc


In [12]:
ytd_data.loc[ytd_data['Short_Desc'].isnull(), 'Short_Desc'] = 'unsure'

In [18]:
reocurring_list = ['progressive']
ytd_data.loc[ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'Yes'
ytd_data.loc[~ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'No'

mapping = {
    'Housing': [],
    'Transportation': [],
    'Food': [],
    'Insurance': ['progressive'],
    'Utilities': [],
    'Medical': [],
    'Government': [],
    'Savings': ['income'],
    'Personal_Spending': ['school'],
    'Recreation_Entertainment': [],
    'Misc': ['transfer','refund'],
    'People': ['family'],
    'Payment_and_Interest': ['interest']
}

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Short_Desc.isin(v), 'Category_2'] = k

ytd_data[ytd_data['Category_2'].isnull()].sort_values(by='Description', ascending=False).sort_values(by='Short_Desc',
                                                                                                     ascending=False)

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date,Short_Desc,Reoccurring_Flag,Category_2


In [19]:
x = ytd_data.groupby(['Reoccurring_Flag']).size().reset_index(name='count')

x.sort_values(by='count', ascending=False)

,Reoccurring_Flag,count
0,No,101
1,Yes,7


In [20]:
ytd_sorted = ytd_data.sort_values(by='Trans. Date').reset_index(drop=True)

# Ensure 'Sort_Date' is also in datetime format for the entire DataFrame
ytd_sorted['Trans. Date'] = pd.to_datetime(ytd_sorted['Trans. Date'])
ytd_sorted['Year'] = ytd_sorted['Trans. Date'].dt.year
ytd_sorted['Counter'] = ytd_sorted.groupby('Short_Desc').cumcount() + 1
ytd_sorted


,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,Year,Counter
0,2025-01-01,To Checking - 8930,Withdrawal,-450.00,4625.54,Posted,sofi_savings,2025-01-01,transfer,No,Misc,2025,1
1,2025-01-07,Nationwide,Direct Deposit,2101.68,6727.22,Posted,sofi_savings,2025-01-01,income,No,Savings,2025,1
2,2025-01-07,To Checking - 8930,Withdrawal,-1634.00,5093.22,Posted,sofi_savings,2025-01-01,transfer,No,Misc,2025,2
3,2025-01-12,From Checking - 8930,Deposit,1000.00,6093.22,Posted,sofi_savings,2025-01-01,transfer,No,Misc,2025,3
4,2025-01-13,To Checking - 8930,Withdrawal,-155.00,5938.22,Posted,sofi_savings,2025-01-01,transfer,No,Misc,2025,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2025-10-14,Nationwide,Direct Deposit,3140.74,7147.16,Posted,sofi_savings,2025-10-01,income,No,Savings,2025,22
104,2025-10-16,DISCOVER,Direct Payment,-1004.32,6142.84,Posted,sofi_savings,2025-10-01,transfer,No,Misc,2025,62
105,2025-10-18,To Checking - 8930,Withdrawal,-1000.00,5142.84,Posted,sofi_savings,2025-10-01,transfer,No,Misc,2025,63
106,2025-10-21,To Checking - 8930,Withdrawal,-1000.00,4142.84,Posted,sofi_savings,2025-10-01,transfer,No,Misc,2025,64


In [21]:
columns = ['Trans. Date', 'Description', 'Amount', 'Category', 'Short_Desc', 'Reoccurring_Flag',
                     'Category_2', 'Date', 'Year', 'Sort_Date', 'CumAmount', 'Counter', 'Account']
ytd_sorted['Category'] = 'NA'
ytd_sorted['Sort_Date'] = ytd_sorted['Trans. Date']

sofi_svaings = ytd_sorted[columns]

In [22]:
sofi_svaings.to_excel('D:\\Sean\\Documents\\Personal\\Budget\\2025_output\\sofi_savings_Curated_Bills.xlsx')